In [2]:
from __future__ import division
from time import time
import re
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import xgboost as xgb
from sklearn.cross_validation import StratifiedKFold
from sklearn.cross_validation import cross_val_score
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.feature_selection import SelectFromModel, VarianceThreshold
import xgboost as xgb
from sklearn.cross_validation import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import normalize
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from scipy.sparse import lil_matrix

from sklearn.cross_validation import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import normalize

C:\Users\KrystalU\Anaconda2\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
print('loading files...')
train = pd.read_csv('C:/Users/KrystalU/Documents/Data/Driver/train.csv', na_values=-1)
test = pd.read_csv('C:/Users/KrystalU/Documents/Data/Driver//test.csv', na_values=-1)

loading files...


In [3]:
unwanted = train.columns[train.columns.str.startswith('ps_calc_')]
train = train.drop(unwanted, axis=1)
test = test.drop(unwanted, axis=1)
origin_cols = list(set(train.columns) - set(['id', 'target']))

In [11]:
def gini(actual, pred, cmpcol=0, sortcol=1):
    assert (len(actual) == len(pred))
    tmp = np.asarray(np.c_[actual, pred, np.arange(len(actual))], dtype=np.float)
    tmp = tmp[np.lexsort((tmp[:, 2], -1 * tmp[:, 1]))]
    totalLosses = tmp[:, 0].sum()
    giniSum = tmp[:, 0].cumsum().sum() / totalLosses

    giniSum -= (len(actual) + 1) / 2.
    return giniSum / len(actual)

def gini_normalized(a, p):
    return gini(a, p) / gini(a, a)


def gini_xgb(preds, dtrain):
    labels = dtrain.get_label()
    gini_score = gini_normalized(labels, preds)
    return 'gini', gini_score

params = {
    'min_child_weight': 9,
    'objective': 'binary:logistic',
    'max_depth': 7,
    'max_delta_step': 1.5,
    'colsample_bytree': 0.6,
    'subsample': 0.8,
    'eta': 0.01,
    'gamma': 0.8,
    'num_boost_round': 1000,
    'random_state': 42
    }

X = train.drop(['id', 'target'], axis=1)
y = train['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify = y)
d_train = xgb.DMatrix(X_train, y_train)
d_valid = xgb.DMatrix(X_test, y_test)
watchlist = [(d_train, 'train'), (d_valid, 'valid')]

start = time()
xgb_model = xgb.train(params, d_train, 1600, watchlist, early_stopping_rounds=100,
                      feval=gini_xgb, maximize=True, verbose_eval=100)
print 'Training time: {} mins.'.format((time() - start) / 60)

[0]	train-error:0.036447	valid-error:0.036449	train-gini:0.009202	valid-gini:-0.005398
Multiple eval metrics have been passed: 'valid-gini' will be used for early stopping.

Will train until valid-gini hasn't improved in 100 rounds.
[100]	train-error:0.036447	valid-error:0.036449	train-gini:0.297619	valid-gini:0.260624
[200]	train-error:0.036447	valid-error:0.036449	train-gini:0.314296	valid-gini:0.263077
[300]	train-error:0.036445	valid-error:0.036449	train-gini:0.332032	valid-gini:0.267259
[400]	train-error:0.036441	valid-error:0.036449	train-gini:0.351214	valid-gini:0.271633
[500]	train-error:0.03643	valid-error:0.036449	train-gini:0.370415	valid-gini:0.275676
[600]	train-error:0.036422	valid-error:0.036466	train-gini:0.389309	valid-gini:0.278737
[700]	train-error:0.036422	valid-error:0.036466	train-gini:0.40526	valid-gini:0.280452
[800]	train-error:0.036418	valid-error:0.036474	train-gini:0.421215	valid-gini:0.281362
[900]	train-error:0.036416	valid-error:0.036474	train-gini:0.4328

In [12]:
XGB_FILE = 'xgb.txt'
xgb_model.dump_model(XGB_FILE)

In [3]:
XGB_FILE = 'xgb.txt'
f = open(XGB_FILE,'r')
feature_set = set()
for line in f.readlines():
    if '<' in line:             #feature line
        line = line.split(':')[1].strip()
        feature_re = re.match('\[(.*)?\]', line)
        info = feature_re.group(0)              #should be only one group
        info = re.sub('\[|\]','',info)
        feature_set.add(info)

In [4]:
feature_set = list(feature_set)

In [8]:
fea_map = pd.DataFrame(feature_set)
fea_map.columns = ['feature']
fea_map['key'] = fea_map['feature'].apply(lambda x: x.split('<')[0])
fea_map['value'] = fea_map['feature'].apply(lambda x: float(x.split('<')[1]))

In [ ]:
fea_map = fea_map['']
fea_map.to_csv('fea_map.csv',index=False)

In [6]:
start = time()
#feature encoder
for j, info in enumerate(feature_set[:400]):
    key = info.split("<")[0].strip()
    value = float(info.split("<")[1].strip())
    train[info] = train[key].apply(lambda x: 1 if x<value else 0)
    test[info] = test[key].apply(lambda x: 1 if x<value else 0)
print 'Feature adding time: {} mins'.format((time()-start)/60)

Feature adding time: 3.66776666641 mins


In [7]:
#remove original feature
train = train.drop(origin_cols, axis=1)
test = test.drop(origin_cols, axis=1)

In [8]:
sub = pd.DataFrame()
sub['id'] = test['id']
sub['target'] = np.zeros_like(test['id'])
test = test.drop('id', axis=1)

In [9]:
train = train.drop('id',axis=1)

In [15]:
StratifiedKFold?

In [11]:
fea = list(train.columns)
fea.remove('target')
X = train[fea].values
y = train['target']
kfold = 5  # need to change to 5
skf = StratifiedKFold(y, n_folds=kfold, random_state=42)
clf = LogisticRegression(penalty='l1', C=0.25, random_state=1580)
for i, (train_index, test_index) in enumerate(skf):
    print('LR kfold: {}  of  {} : '.format(i + 1, kfold))
    X_train, X_valid = X[train_index], X[test_index]
    y_train, y_valid = y[train_index], y[test_index]
    clf.fit(X_train, y_train) 
    sub['target'] += clf.predict_proba(test[fea])[:,1] / kfold

LR kfold: 1  of  5 : 
LR kfold: 2  of  5 : 
LR kfold: 3  of  5 : 
LR kfold: 4  of  5 : 
LR kfold: 5  of  5 : 


In [12]:
sub.head()

,id,target
0,0,0.027132
1,1,0.025871
2,2,0.034203
3,3,0.022911
4,4,0.043251


In [13]:
sub.to_csv('sub13_lr.csv',index=False)